In [29]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Embedding, Reshape, Dropout, Dense, merge, Dot
from keras.models import Sequential 
from keras.optimizers import Adamax
from keras.callbacks import EarlyStopping, ModelCheckpoint

### Model

In [24]:
class CFModel(Sequential):

    def __init__(self, n_users, m_items, k_factors, **kwargs):
        P = Sequential()
        P.add(Embedding(n_users, k_factors, input_length=1))
        P.add(Reshape((k_factors,)))
        Q = Sequential()
        Q.add(Embedding(m_items, k_factors, input_length=1))
        Q.add(Reshape((k_factors,)))
        super(CFModel, self).__init__(**kwargs)
        self.add(Merge([P, Q], mode='dot', dot_axes=1))

    def rate(self, user_id, item_id):
        return self.predict([np.array([user_id]), np.array([item_id])])[0][0]


class DeepModel(Sequential):

    def __init__(self, n_users, m_items, k_factors, p_dropout=0.1, **kwargs):
        P = Sequential()
        P.add(Embedding(n_users, k_factors, input_length=1))
        P.add(Reshape((k_factors,)))
        Q = Sequential()
        Q.add(Embedding(m_items, k_factors, input_length=1))
        Q.add(Reshape((k_factors,)))
        super(DeepModel, self).__init__(**kwargs)
        self.add(Merge([P, Q], mode='concat'))
        self.add(Dropout(p_dropout))
        self.add(Dense(k_factors, activation='relu'))
        self.add(Dropout(p_dropout))
        self.add(Dense(1, activation='linear'))

    def rate(self, user_id, item_id):
        return self.predict([np.array([user_id]), np.array([item_id])])[0][0]

### Load data

In [7]:
K_FACTORS = 120
RNG_SEED = 1446557

ratings = pd.read_csv('foo2.csv')
ratings.drop(['index'],axis=1, inplace=True)
ratings.head()

,userid,movieid,rating
0,44,1,4
1,61,1,3
2,67,1,4
3,72,1,3
4,86,1,5


In [9]:
from sklearn.preprocessing import LabelEncoder
user_enc = LabelEncoder()
movie_enc = LabelEncoder()
user_enc.fit(ratings.userid.unique())
movie_enc.fit(ratings.movieid.unique())
n_users = len(user_enc.classes_)
n_movies = len(movie_enc.classes_)

In [10]:
ratings.userid = user_enc.transform(ratings.userid)
ratings.movieid = movie_enc.transform(ratings.movieid)
ratings.head()

,userid,movieid,rating
0,43,0,4
1,60,0,3
2,66,0,4
3,71,0,3
4,85,0,5


In [14]:
shuffled_ratings = ratings.sample(frac=1., random_state=RNG_SEED)
Users = shuffled_ratings['userid'].values
print('Users:', Users, ', shape =', Users.shape)
Movies = shuffled_ratings['movieid'].values
print('Movies:', Movies, ', shape =', Movies.shape)
Ratings = shuffled_ratings['rating'].values
print('Ratings:', Ratings, ', shape =', Ratings.shape)

Users: [5118 8984 4603 ... 1635 9523 4863] , shape = (1176952,)
Movies: [754 465 620 ... 910 819 105] , shape = (1176952,)
Ratings: [5 2 5 ... 4 4 3] , shape = (1176952,)


### Define model

In [25]:
max_userid = ratings['userid'].drop_duplicates().max()+1
max_movieid = ratings['movieid'].drop_duplicates().max()+1

model = CFModel(max_userid, max_movieid, K_FACTORS)
model.compile(loss='mse', optimizer='adamax')

TypeError: 'module' object is not callable

### Train model

In [17]:
callbacks = [EarlyStopping('val_loss', patience=2), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE, save_best_only=True)]
history = model.fit([Users, Movies], Ratings, nb_epoch=30, validation_split=.1, verbose=2, callbacks=callbacks)

NameError: name 'MODEL_WEIGHTS_FILE' is not defined

### Plot training and validation RMSE

In [19]:
loss = pd.DataFrame({'epoch': [ i + 1 for i in history.epoch ],
                     'training': [ math.sqrt(loss) for loss in history.history['loss'] ],
                     'validation': [ math.sqrt(loss) for loss in history.history['val_loss'] ]})
ax = loss.ix[:,:].plot(x='epoch', figsize={7,10}, grid=True)
ax.set_ylabel("root mean squared error")
ax.set_ylim([0.0,3.0]);

NameError: name 'history' is not defined

### Print best validation RMSE

In [ ]:
min_val_loss, idx = min((val, idx) for (idx, val) in enumerate(history.history['val_loss']))
print 'Minimum RMSE at epoch', '{:d}'.format(idx+1), '=', '{:.4f}'.format(math.sqrt(min_val_loss))